In [1]:
# https://dev.to/1n0t/matplotlib-heatmap-calendar-3kbo

In [2]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import yfinance as yf

from datetime import date
from datetime import datetime

import os

import data as data_

In [3]:
folderPath_rsrc = '/Resources/'
folderPath_results = '/Results/'
cwd = os.getcwd()
path_rsrc = cwd + folderPath_rsrc
path_results = cwd + folderPath_results
# read data
ETFs = pd.read_excel(path_rsrc + "ETFs list.xlsx")

In [4]:
ETFs

Technology Energy Telecom Consumer Discretionary Real Estate Industrials  \
0         AAPB   ACES    FCOM                   AMZD        BBRE        AIRR   
1         AAPD    AMJ     IXP                   AMZU        BLDG        AQWA   
2         AAPU   AMLP     IYZ                    BAD        BYRE        ARKX   
3          AIQ   AMNA     LTL                   BEDZ        CHIR        AWAY   
4         ARKK   AMND    NXTG                   BETZ        DFGR        BDRY   
..         ...    ...     ...                    ...         ...         ...   
163       XPND    NaN     NaN                    NaN         NaN         NaN   
164        XSD    NaN     NaN                    NaN         NaN         NaN   
165        XSW    NaN     NaN                    NaN         NaN         NaN   
166         XT    NaN     NaN                    NaN         NaN         NaN   
167       XWEB    NaN     NaN                    NaN         NaN         NaN   

    Utilities Materials Healthcare Consumer Staples Financials  
0        BLLD      ALUM       AGNG             CHIS       BDCX  
1        CHIU      BATT       ARKG             CLIX       BDCZ  
2        ECLN      CHIM        BBC             EATV       BIZD  
3        EMIF      CIRC        BBH             EATZ       BNKD  
4        FUTY      COPX        BBP             ECON       BNKU  
..        ...       ...        ...              ...        ...  
163       NaN       NaN        NaN              NaN        NaN  
164       NaN       NaN        NaN              NaN        NaN  
165       NaN       NaN        NaN              NaN        NaN  
166       NaN       NaN        NaN              NaN        NaN  
167       NaN       NaN        NaN              NaN        NaN  

[168 rows x 11 columns]

In [5]:
sector = "Energy"
energy = ETFs[sector]
energy.dropna(inplace=True)

In [6]:
energy = energy.to_list()

In [10]:
end_ = date.today()  # yyyy-mm-dd
if end_.day == 31:
    start_ = datetime(end_.year, end_.month - 1, 30)
else:
    start_ = datetime(end_.year, end_.month - 1, end_.day)

# data = yf.download(energy, start=start_, end=end_, interval="1wk")
data_.downloadAllTickers(tickers=energy, start=start_, end=end_)
# data_.downloadTicker(ticker="XT", start=start_, end=end_)

In [8]:
os.getcwd()

'C:\\Users\\Usuario\\Documents\\GitHub\\Desempe-o-Sectorial\\ETFs'

In [11]:
cwd = os.getcwd()
folderPath_rsrc = '/Resources/'
path_results = cwd + folderPath_rsrc
data = pd.read_pickle(f"{path_results}{energy[0]}")

In [13]:
data.head()

date   open    high     low  close  adjusted_close  volume
0  2023-10-02  36.39  36.390  34.720  34.95           34.95   26400
1  2023-10-03  34.50  34.500  33.522  33.69           33.69   27900
2  2023-10-04  33.82  34.260  33.255  34.13           34.13  372100
3  2023-10-05  33.75  33.750  32.700  32.76           32.76   54800
4  2023-10-06  32.32  33.645  32.200  33.37           33.37  128600

In [ ]:
if end_.day == 31:
    start_ = datetime(end_.year, end_.month - 1, 30)

In [ ]:
df = data.Close
df

## Calcular retornos

In [ ]:
returns = df.pct_change()
returns

In [ ]:
values = returns.iloc[-1].sort_values(ascending=False)
val_max = values.max()
val_min = values.min()
values

In [ ]:
values.values

In [ ]:
test = [round(i*100,2) for i in values]
test[:10]

In [ ]:
length = len(test)
# the idea is to shape an a X b matrix
a = int(np.sqrt(length))
b = int(np.ceil(length/a))
m_length = a * b                # matrix "length"

if m_length >= length:
    # compute how many elements need to be add to the array
    diff = abs(length - m_length)
    
    add =np.full(diff, np.nan)
    new_array = np.append(test, add)
    new_array = new_array.reshape(a, b)
    
    # add names to the plot
    names = values.index.to_list()
    add_names =np.full(diff, '---')
    new_names = np.append(names, add_names)
    new_names = new_names.reshape(a, b)

In [ ]:
new_array

In [ ]:
valores = new_array
from  matplotlib.colors import LinearSegmentedColormap
cmap_=LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N=256) 

# "RdYlGn"
fig, ax = plt.subplots()

# Especificamos paleta de colores a usar y rango de valores a representar.
ax.imshow(valores, cmap=cmap_, vmin=val_min*100, vmax=val_max*100)
# borrar axis labels
ax.set_yticklabels([])
ax.set_xticklabels([])

# Reducimos la longitud de las marcas a 0 para que no sean visibles
ax.tick_params(axis=u'both', which=u'both', length=0)
for lado in ['left', 'right', 'bottom', 'top']:
    ax.spines[lado].set_visible(False)

# # Indicamos las posiciones donde dibujaremos la rejilla  
xmin,xmax = ax.get_xlim()
ymin,ymax = ax.get_ylim()

ax.set_yticks(np.arange(ymax, ymin+1), minor=True)
ax.set_xticks(np.arange(xmin, xmax+1), minor=True)
# Dibujamos la rejilla de color blanco para que actue como separador.
ax.grid(which='minor', color='w', linestyle='-', linewidth=2)

for i in range(new_array.shape[0]):
    for j in range(new_array.shape[1]):
        # print(i, j, valores[i,j])
        text = ax.text(j, i, str(new_names[i, j]) + '\n' + str(valores[i, j])+'%',
                       ha="center", va="center", color="k", size=6)

ax.set_title(f"{sector}\n")
        
plt.savefig(f"{sector}.png")
plt.show()
